In [2]:
install.packages("raster")
install.packages("RItools")

Warning message in install.packages("raster"):
“installation of package ‘raster’ had non-zero exit status”
Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [3]:
#!/usr/bin/env Rscript

# This global processing script PART II is derived from the global processing notebook 
#the input can be the iso3 code (3-character) for one or multiple countries 

#mamba install -c conda-forge r-terra r-optmatch r-sp r-rgdal r-sf r-rgeos r-dplyr r-plyr r-ggplot2 r-raster r-mapview r-stringr r-maptools r-gridExtra r-lattice r-MASS r-foreach r-doParallel r-rlang r-tidyr r-magrittr r-viridis r-ggmap r-Hmisc r-hrbrthemes r-spatialEco r-bit64 r-randomForest r-modelr

#mamba install -c conda-forge r-ranger r-caret

#mamba install -c conda-forge r-rgeos r-ritools

options(warn=-1)
options(dplyr.summarise.inform = FALSE)

packages <- c("sp","rgdal","sf","rgeos","dplyr","plyr","ggplot2","raster","mapview","stringr",
              "maptools","gridExtra","lattice","MASS","foreach","optmatch","doParallel","RItools","gdalUtils",
              "rlang","tidyr","magrittr","viridis","ggmap","Hmisc","hrbrthemes","spatialEco","bit64","randomForest", "modelr","ranger","caret")
package.check <- lapply(packages, FUN = function(x) {
  suppressPackageStartupMessages(library(x, character.only = TRUE))
})

args = commandArgs(trailingOnly=TRUE)
if (length(args)==0) {
  stop("At least one argument must be supplied (input file).n", call.=FALSE)
} else if (length(args)>=1) {
  
  iso3 <- args[1]  #country to process
  gediwk <- args[2]   #the # of weeks GEDI data to use
  mproc <- as.integer(args[3])#the number of cores to use for macthing 
}

In [15]:
iso3 <- "SEN"
gediwk<-24
mproc <-1

cat("Step 0: Loading global variables to process country", iso3,"with GEDI data until week", gediwk, "\n")

f.path <- cat("~/GEDI_PA/Matching_Layers/SEN/")
ecoreg_key <- read.csv(paste("~/GEDI_PA/wwf_ecoregions_key.csv",sep=""))
allPAs <- readRDS(paste("~/GEDI_PA/Matching_Layers/SEN/SEN_PA_poly.rds",sep=""))
MCD12Q1 <- raster(paste("~/GEDI_PA/GEDI_ANCI_PFT_r1000m_EASE2.0_UMD_v1_projection_defined_6933.tif",sep=""))
projection(MCD12Q1) <- sp::CRS(paste("+init=epsg:",6933,sep=""))
world_region <- raster(paste("~/GEDI_PA/GEDI_ANCI_CONTINENT_r1000m_EASE2.0_UMD_v1_revised_projection_defined_6933.tif",sep=""))
projection(world_region) <- sp::CRS(paste("+init=epsg:",6933,sep=""))
adm <- readOGR(paste("~/GEDI_PA/Matching_Layers/SEN/SEN_admin.geojson"),verbose=FALSE)
adm_prj <- spTransform(adm, "+init=epsg:6933") 
load("~/GEDI_PA/rf_noclimate.rdata")
source("~/GEDI_PA/matching_func.r")
# flag <- "don't ran extraction"
flag <- "run all"
# flag <- "run remaining"

Step 0: Loading global variables to process country SEN with GEDI data until week 24 
~/GEDI_PA/Matching_Layers/SEN/

In [8]:
matched_all <- list.files(paste("~/GEDI_PA/Matching_Layers/SEN/WDPA_matching_points/",iso3,"_wk",gediwk,sep=""), pattern=".RDS", full.names = FALSE) 

In [9]:
id_pa <- 865
#GEDI_PA/Matching_Layers/SEN/WDPA_matching_points/SEN_wk24/SEN_pa_2332_matching_results_wk24.RDS
matched <- readRDS(paste("~/GEDI_PA/Matching_Layers/SEN/WDPA_matching_points/SEN_wk24/",iso3,"_pa_", id_pa,"_matching_results_wk",gediwk,".RDS", sep=""))
print(matched)

# A tibble: 15,116 × 21
     lat   lon   UID status land_cover wwfbiom         wwfecoreg elevation slope
   <dbl> <dbl> <int> <lgl>  <fct>      <fct>           <fct>         <dbl> <dbl>
 1  13.0 -13.5    45 TRUE   l1_forest  Tropical & Sub… West Sud…        35 0.658
 2  12.7 -13.3   141 TRUE   l1_forest  Tropical & Sub… Guinean …        62 1.43 
 3  12.9 -13.4   157 TRUE   l1_forest  Tropical & Sub… Guinean …        49 0.613
 4  13.1 -13.5   171 TRUE   l1_forest  Tropical & Sub… West Sud…        20 0.163
 5  12.7 -13.3   178 TRUE   l1_forest  Tropical & Sub… Guinean …        86 1.29 
 6  13.0 -13.5   215 TRUE   l1_forest  Tropical & Sub… West Sud…        19 0.176
 7  12.8 -13.4   240 TRUE   l1_forest  Tropical & Sub… Guinean …        70 0.744
 8  12.7 -13.3   268 TRUE   l1_forest  Tropical & Sub… Guinean …       120 0.812
 9  12.8 -13.4   280 TRUE   l1_forest  Tropical & Sub… Guinean …        47 0.470
10  12.7 -13.3   315 TRUE   l1_forest  Tropical & Sub… Guinean …        95 0.835
# ℹ 

In [10]:
#---------------STEP5. GEDI PROCESSING - using GEDI shots to extract the treatment/control status, also extract the MODIS PFT for AGB prediction---------------- 
# if (file.exists(paste(f.path,"WDPA_GEDI_extract/",iso3,"_wk",gediwk,"/",iso3,"_gedi_extracted_matching_wk",gediwk,".RDS", sep=""))){
cat(paste("Step 5: Performing WK ",gediwk,"GEDI extraction for", iso3,"\n"))
#matched_all <-read.csv(paste(f.path,"WDPA_extract4_residual_PAs/", iso3, "_wk_", gediwk, "_null_matches_rerun.csv",sep="")) 
matched_all<-list.files(paste("~/GEDI_PA/Matching_Layers/SEN/WDPA_matching_points/",iso3,"_wk",gediwk,sep=""), pattern=".RDS", full.names = FALSE)
# readr::parse_number() %>% unique()
#list.files(paste(f.path,"WDPA_matching_results/",iso3,"_wk",gediwk,sep=""), pattern=".RDS", full.names = FALSE)
registerDoParallel(3)
matched_PAs <- foreach(this_rds=matched_all, .combine = c, .packages=c('sp','magrittr', 'dplyr','tidyr','raster')) %dopar% {   #non-NA matched results
  matched_PAs=c()
  print(this_rds)
  if(nchar(iso3)>3){
    id_pa <- this_rds %>% str_split("_") %>% unlist %>% .[4]  
  } else {
    id_pa <- this_rds %>% str_split("_") %>% unlist %>% .[3]
  }
  matched <- readRDS(paste("~/GEDI_PA/Matching_Layers/SEN/WDPA_matching_points/SEN_wk24/",iso3,"_pa_", id_pa,"_matching_results_wk",gediwk,".RDS", sep=""))
  if(!is.null(matched)){
    if(nrow(matched)!=0){
      matched_PAs=c(matched_PAs,this_rds) 
    }
  }else {
    # print(this_rds)
    matched_PAs=matched_PAs
  }
  return(matched_PAs)
}
stopImplicitCluster()

Step 5: Performing WK  24 GEDI extraction for SEN 


In [11]:
if(flag=="run all"){  #determine how many PAs to run the extraction process
  matched_PAs <- matched_PAs
  cat("Step 5: runing extraction on all", length(matched_PAs),"of non-NA matched results in", iso3,"\n")
} else if (flag=="run remaining"){
  pattern1 = c(paste("wk",gediwk,sep=""),"RDS")
  extracted_PAid <- list.files(paste("~/GEDI_PA/Matching_Layers/SEN/WDPA_GEDI_extract4/",iso3,"_wk",gediwk,"/",sep=""), full.names = F, pattern=paste0(pattern1, collapse="|"))%>%
    readr::parse_number() %>% unique()
  matched_PA_id <- matched_PAs %>% readr::parse_number()
  runPA_id <- matched_PA_id[!(matched_PA_id %in% extracted_PAid)]
  if (length(runPA_id)>0){
    Pattern2 <-  paste(runPA_id, collapse="|")
    runPA <-  matched_PAs[grepl(Pattern2,matched_PAs)]
    # runPA_ind <- str_detect(matched_PAs, paste(runPA_id, collapse = "|"))
    matched_PAs <-runPA
  } else {
    matched_PAs <- NULL
    cat("Step 5 already done for", iso3, "\n")
  }
}

Step 5: runing extraction on all 108 of non-NA matched results in SEN 


In [12]:
registerDoParallel(cores=round(mproc))
getDoParWorkers()
startTime <- Sys.time()

[1] 1

In [13]:
print(matched)

# A tibble: 15,116 × 21
     lat   lon   UID status land_cover wwfbiom         wwfecoreg elevation slope
   <dbl> <dbl> <int> <lgl>  <fct>      <fct>           <fct>         <dbl> <dbl>
 1  13.0 -13.5    45 TRUE   l1_forest  Tropical & Sub… West Sud…        35 0.658
 2  12.7 -13.3   141 TRUE   l1_forest  Tropical & Sub… Guinean …        62 1.43 
 3  12.9 -13.4   157 TRUE   l1_forest  Tropical & Sub… Guinean …        49 0.613
 4  13.1 -13.5   171 TRUE   l1_forest  Tropical & Sub… West Sud…        20 0.163
 5  12.7 -13.3   178 TRUE   l1_forest  Tropical & Sub… Guinean …        86 1.29 
 6  13.0 -13.5   215 TRUE   l1_forest  Tropical & Sub… West Sud…        19 0.176
 7  12.8 -13.4   240 TRUE   l1_forest  Tropical & Sub… Guinean …        70 0.744
 8  12.7 -13.3   268 TRUE   l1_forest  Tropical & Sub… Guinean …       120 0.812
 9  12.8 -13.4   280 TRUE   l1_forest  Tropical & Sub… Guinean …        47 0.470
10  12.7 -13.3   315 TRUE   l1_forest  Tropical & Sub… Guinean …        95 0.835
# ℹ 

In [30]:
foreach(this_rds=matched_PAs, .combine = foreach_rbind, .packages=c('sp','magrittr', 'dplyr','tidyr','raster')) %dopar% {
  cat("Extracting for no. ", match(this_rds,matched_PAs),"pa out of", length(matched_PAs),"\n")
  if(nchar(iso3)>3){
    id_pa <- this_rds %>% str_split("_") %>% unlist %>% .[4]  
  } else {
    id_pa <- this_rds %>% str_split("_") %>% unlist %>% .[3]
  }
  matched <- readRDS(paste("~/GEDI_PA/Matching_Layers/SEN/WDPA_matching_points/",iso3,"_wk",gediwk,"/",iso3,"_pa_", "2332","_matching_results_wk",gediwk,".RDS", sep=""))
  if (is.null(matched)==TRUE  | nrow(matched)==0) {
    cat("Matched result is null for PA", id_pa, "quitting...\n")
  } else if (!is.null(matched)==TRUE){
    mras  <- tryCatch(matched2ras(matched),
                      error=function(cond){
                        message(cond)
                        cat("Matched result is likely null for country", iso3,"pa", id_pa, "dimension of the match is", dim(matched),"\n")
                        # writeLines("Matched results is likely null for country", paste(f.path,"WDPA_log/",iso3,"_log_matching.txt", sep=""))
                        return(NULL)}) #convert the macthed df to a raster stack 
    if(table(mras$status[])[2]==0 | table(mras$status[])[1]==0 | is.null(mras)){
      cat("Rasterized results unbalanced for PA", id_pa, "quitting...\n")
    } else {
      startTime <- Sys.time()
      # iso_matched_gedi<- extract_gedi(matched=matched, mras = mras)  #run filtered csvs on mras for extarction
      tElapsed <- Sys.time()-startTime
    iso_matched_gedi<- matched
      cat(tElapsed, "for extracting all PAs in", iso3,"\n")
      # iso_matched_gedi_sub <- iso_matched_gedi %>%
      #   dplyr::select("pa_id","shot_number","status","DESIG_ENG.x","wwfbiom","wwfecoreg","PADDD","pft","region","lon_lowestmode","lat_lowestmode",
      #                 "rh_010","rh_020", "rh_030", "rh_040",  "rh_050",  "rh_060" , "rh_070","rh_075",  "rh_080",  "rh_090",  "rh_098", "AGBD","cover","pai","REGION","PFT")  #write to individual country folder
      if (length(unique(iso_matched_gedi$wwfbiom)) >1){
        pabiome <- iso_matched_gedi$wwfbiom %>% unique() %>% gsub('\\b(\\pL)\\pL{2,}|.','\\U\\1',.,perl = TRUE)%>% str_c( collapse = "+")
      } else if (length(unique(iso_matched_gedi$wwfbiom))==1){
        pabiome <- iso_matched_gedi$wwfbiom %>% unique() %>% gsub('\\b(\\pL)\\pL{2,}|.','\\U\\1',.,perl = TRUE)
      } else {
        pabiome <- iso_matched_gedi$wwfbiom %>% unique()
      }
      # papaddd <- unique(iso_matched_gedi$PADDD) %>% getmode()
      continent <- unique(iso_matched_gedi$region) %>% getmode()

      dir.create(file.path(paste("~/GEDI_PA/Matching_Layers/SEN/WDPA_GEDI_extract4/",iso3,"_wk",gediwk,"/",sep="")))
      saveRDS(iso_matched_gedi, file=paste("~/GEDI_PA/Matching_Layers/SEN/WDPA_GEDI_extract4/SEN_wk24/",iso3,"_pa_", id_pa,"_gedi_wk_",gediwk,"_conti_","biome_",pabiome,".RDS", sep=""))
      cat(id_pa,"in",iso3,"results is written to dir\n")
      write.csv(iso_matched_gedi, file=paste("~/GEDI_PA/Matching_Layers/SEN/WDPA_GEDI_extract4/SEN_wk24/","SEN_pa_", id_pa,"_iso_matched_gedi_sub_wk_",gediwk,".csv", sep=""))
    }
  }
  return(NULL)
}

stopImplicitCluster()
tElapsed <- Sys.time()-startTime
cat(tElapsed, "for extracting all PAs in", iso3,"\n")
cat("Done GEDI extraction for pa in ",iso3,"\n")    

Extracting for no.  1 pa out of 108 
SEN converting the matched csv to a raster stack for extraction
5.483627e-06 for extracting all PAs in SEN 
2332 in SEN results is written to dir
Extracting for no.  2 pa out of 108 
SEN converting the matched csv to a raster stack for extraction
5.483627e-06 for extracting all PAs in SEN 
2333 in SEN results is written to dir
Extracting for no.  3 pa out of 108 
SEN converting the matched csv to a raster stack for extraction
5.483627e-06 for extracting all PAs in SEN 
2578 in SEN results is written to dir
Extracting for no.  4 pa out of 108 
SEN converting the matched csv to a raster stack for extraction
6.67572e-06 for extracting all PAs in SEN 
2580 in SEN results is written to dir
Extracting for no.  5 pa out of 108 
SEN converting the matched csv to a raster stack for extraction
5.483627e-06 for extracting all PAs in SEN 
3044 in SEN results is written to dir
Extracting for no.  6 pa out of 108 
SEN converting the matched csv to a raster stack 

NULL

In [31]:
stopImplicitCluster()
tElapsed <- Sys.time()-startTime
cat(tElapsed, "for extracting all PAs in", iso3,"\n")
cat("Done GEDI extraction for pa in ",iso3,"\n")    

1.109392 for extracting all PAs in SEN 
Done GEDI extraction for pa in  SEN 
